In [2]:
import matplotlib.pylab as plt
from astropy.io import fits as pyfits
import sys
import numpy as np
import matplotlib as mplt
import scipy.signal
import os
import shutil

In [15]:
def headerSearch(filelist, SEARCHKEY, SEARCHVAL):
    """
    Scans list of file names to display exposure times and object types from HDU header
        filelist:
        SEARCHKEY:
        SEARCHVAL:
        Returns: None
    """
    fh = open(filelist, "r")

    for filename in fh:
        hdul = pyfits.open(filename.rstrip())  # rstrip is to remove carriage return
        head = hdul[0].header
        #print(filename, head[SEARCHKEY])
        try:
            #if head[SEARCHKEY] == SEARCHVAL:
            if head[SEARCHKEY].find(SEARCHVAL)>-1:
                print(" file = {}, OBJECT = {}, EXPOSURE = {}".format(filename.rstrip(), head["OBJECT"],
                                                                      head["EXPTIME"]))
        except:
            pass
        try:
            if head["OBJECT"] == "DARK" or head["OBJECT"] == "BIAS":
                print(" file = {}, OBJECT = {}, EXPOSURE = {}".format(filename.rstrip(), head["OBJECT"],
                                                                      head["EXPTIME"]))
        except:
            pass

        hdul.close()

    fh.close()

In [16]:
def subtractImg(fout, fitsToSubtract, filelist):
    '''takes(fout,fitsToSubtract,filelist)  '''

    hdulref = pyfits.open(fitsToSubtract)
    head = hdulref[0].header
    refdata = hdulref[0].data
    hdulref.close()

    fh = open(filelist, "r")
    iter = 0
    for line in fh:
        fpath = line.rstrip()
        hdul = pyfits.open(fpath)

        head = hdul[0].header
        imgdata = hdul[0].data

        Ndata = imgdata.size

        Nref = refdata.size

        if not Ndata == Nref:
            print("Mismatch between data {} and reference {}".format(Ndata, Nref))
            sys.exit()

        NY, NX = imgdata.shape

        corrected = imgdata * 0
        for j in range(NY):
            for i in range(NX):
                if refdata[j][i] > imgdata[j][i]:
                    corrected[j][i] = 0
                else:
                    corrected[j][i] = imgdata[j][i] - refdata[j][i]

        head.set('comment', 'Subtracted {} from file {}'.format(fitsToSubtract, fpath))
        hdul[0].data = corrected

        hdul.writeto(fout + "-" + repr(iter).zfill(6) + ".fits")
        hdul.close()
        iter += 1

In [17]:
def medianCombine(fout, filelist):
    '''Median combine files in file list
       takes (fout,filelist)'''

    fh = open(filelist, "r")
    NFILES = 0
    for line in fh: NFILES += 1
    fh.seek(0)
    print("Number of files is {}".format(NFILES))

    hdularr = []

    hdul = pyfits.open(fh.readline().rstrip())
    hdularr.append(hdul)

    imgdata0 = hdul[0].data
    NY, NX = imgdata0.shape
    print(NY, NX)

    imgcube = np.zeros([NFILES, NY, NX])
    print(imgcube[0].shape)

    i = 1
    names = []
    for line in fh:
        names.append(line.rstrip())
        hdul = pyfits.open(line.rstrip())
        hdularr.append(hdul)
        data = hdul[0].data
        print(data.shape)
        imgcube[i][0:-1][0:-1] = data[0:-1][0:-1]
        i += 1
    fh.close()

    # check lengths
    Ndata0 = imgcube.size
    for i in range(1, NFILES):
        Ndatai = imgcube.size
        print(Ndatai)
        if not Ndata0 == Ndatai:
            print("Data length mismatch between file 0 and file {}".format(i))
            sys.exit()

    hdulout = hdularr[NFILES // 2]
    head = hdulout[0].header
    meddata = imgdata0 * 0.

    for j in range(NY):
        for i in range(NX):
            cubeslice = np.zeros(NFILES)
            for s in range(NFILES): cubeslice[s] = imgcube[s][j][i]
            meddata[j][i] = np.median(cubeslice)

    hdulout[0].data = meddata
    head.set('comment',
             'Median combined from {} files. Header from central image ({}).'.format(NFILES, names[NFILES // 2]))

    hdulout.writeto(fout)
    for i in range(NFILES): hdularr[i].close()

In [18]:
def imageShift(fout, DNX, DNY, filelist="shift.list"):
    '''Register images by shifting in X and Y.
       takes (fout,DNX,DNY,filelist="shift.list") '''

    fh = open(filelist, "r")
    iter = 0
    for line in fh:
        fpath = line.rstrip()
        hdul = pyfits.open(fpath)

        head = hdul[0].header
        imgdata = hdul[0].data
        NY, NX = imgdata.shape

        ind1d = DNX + NX * DNY

        shifted = np.reshape(np.roll(np.ravel(imgdata), ind1d), imgdata.shape)

        head.set('comment', 'imageShift')
        head.set('comment', 'original file: ' + fpath)
        hdul[0].data = shifted

        hdul.writeto(fout + "-" + repr(iter).zfill(6) + ".fits")
        hdul.close()
        iter += 1
    fh.close()

In [19]:
def alignByShift(fout, REFFITS, filelist="shift.list"):
    '''Register images by shifting in X and Y. Uses cross-correlation to find displacement
       takes (fout,REFFITS,filelist="shift.list") '''

    hdulref = pyfits.open(REFFITS)
    refhead = hdulref[0].header
    refdata = hdulref[0].data
    hdulref.close()

    fh = open(filelist, "r")
    iter = 0
    for line in fh:
        fpath = line.rstrip()
        hdul = pyfits.open(fpath)

        head = hdul[0].header
        imgdata = hdul[0].data

        Ndata = len(imgdata)

        Nref = len(refdata)

        if not Ndata == Nref:
            print("Mismatch between data {} and reference {}".format(Ndata, Nref))
            sys.exit()

        print("Starting cross correlation for {}".format(fpath))

        I_imgdata = np.fft.fft(np.ravel(imgdata))
        I_refdata = np.fft.fft(np.ravel(refdata))

        V = I_imgdata.conjugate() * I_refdata

        crosscor = np.fft.ifft(V).real
        ind1d = np.argmax(crosscor)

        shifted = np.reshape(np.roll(np.ravel(imgdata), ind1d), imgdata.shape)

        head.set('comment', 'alignedByShift')
        head.set('comment', 'original file: ' + fpath)
        hdul[0].data = shifted

        hdul.writeto(fout + "-" + repr(iter).zfill(6) + ".fits")
        hdul.close()
        iter += 1
    fh.close()